In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
# six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
# six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    ic(name)
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)

# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)]=(matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0

matrix['delta_price_lag']=matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

ic| name: '!Якутск Орджоникидзе, 56 фран'
ic| name: '!Якутск ТЦ "Центральный" фран'
ic| name: 'Адыгея ТЦ "Мега"'
ic| name: 'Балашиха ТРК "Октябрь-Киномир"'
ic| name: 'Волжский ТЦ "Волга Молл"'
ic| name: 'Вологда ТРЦ "Мармелад"'
ic| name: 'Воронеж (Плехановская, 13)'
ic| name: 'Воронеж ТРЦ "Максимир"'
ic| name: 'Воронеж ТРЦ Сити-Парк "Град"'
ic| name: 'Выездная Торговля'
ic| name: 'Жуковский ул. Чкалова 39м?'
ic| name: 'Жуковский ул. Чкалова 39м²'
ic| name: 'Интернет-магазин ЧС'
ic| name: 'Казань ТЦ "Бехетле"'
ic| name: 'Казань ТЦ "ПаркХаус" II'
ic| name: 'Калуга ТРЦ "XXI век"'
ic| name: 'Коломна ТЦ "Рио"'
ic| name: 'Красноярск ТЦ "Взлетка Плаза"'
ic| name: 'Красноярск ТЦ "Июнь"'
ic| name: 'Курск ТЦ "Пушкинский"'
ic| name: 'Москва "Распродажа"'
ic| name: 'Москва МТРЦ "Афи Молл"'
ic| name: 'Москва Магазин С21'
ic| name: 'Москва ТК "Буденовский" (пав.А2)'
ic| name: 'Москва ТК "Буденовский" (пав.К7)'
ic| name: 'Москва ТРК "Атриум"'
ic| name: 'Москва ТЦ "Ареал" (Беляево)'
ic| name: 'Москва 

In [21]:
matrix.shape

(6639294, 74)

In [22]:
# matrix = matrix_11128050_64_bak.copy()

In [23]:
matrix.shape

(11128050, 64)

In [3]:
# matrix_11128050_64_bak = matrix.copy()

In [24]:
# 月销量（商店类型）
group = matrix.groupby(['date_block_num','shop_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shoptype_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shoptype_avg_item_cnt')
matrix.drop('date_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店类型）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_shoptype_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_shoptype_avg_item_cnt')
matrix.drop('date_item_shoptype_avg_item_cnt', axis=1, inplace=True)

# # 月销量（商店-商品）
# group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
# group.columns = [ 'date_shopitem_avg_item_cnt' ]
# group.reset_index(inplace=True)
# matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
# matrix = lag_features(matrix, [1,2,3,6,12], 'date_shopitem_avg_item_cnt')
# matrix.drop('date_shopitem_avg_item_cnt', axis=1, inplace=True)

In [25]:
matrix.shape

(11128050, 74)

In [26]:
# **********趋势特征 delta_cnt_month_lag ********************
group = matrix.groupby('item_id').agg({'item_cnt_month': 'mean'})
group.columns = ['trend_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['trend_date_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'trend_date_item_avg_cnt_month')
for i in lags:
    matrix['delta_cnt_month_lag_'+str(i)] = (matrix['trend_date_item_avg_cnt_month_lag_' + str(i)] - matrix['trend_item_avg_cnt_month']) / matrix['trend_item_avg_cnt_month']

def select_trend2(row):
    for i in lags:
        if pd.notnull(row['delta_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_cnt_month_lag'] = matrix.apply(select_trend2, axis=1)
matrix['delta_cnt_month_lag'] = matrix['delta_cnt_month_lag'].astype(np.float16)

features_to_drop = ['trend_item_avg_cnt_month','trend_date_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['trend_date_item_avg_cnt_month_lag_'+str(i)]
    features_to_drop += ['delta_cnt_month_lag_'+str(i)]
    matrix.drop(features_to_drop, axis=1, inplace=True)

KeyError: "['trend_item_avg_cnt_month' 'trend_date_item_avg_cnt_month'\n 'trend_date_item_avg_cnt_month_lag_1' 'delta_cnt_month_lag_1'] not found in axis"

In [30]:
matrix.shape

(11128050, 75)

In [32]:
# **********趋势特征 delta2_cnt_month_lag ********************
group = matrix.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_shop_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['shop_id', 'item_id'], how='left')

group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_date_shop_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'qushi_date_shop_item_avg_cnt_month')
for i in lags:
    matrix['delta2_cnt_month_lag_'+str(i)] = (matrix['qushi_date_shop_item_avg_cnt_month_lag_' + str(i)] - matrix['qushi_shop_item_avg_cnt_month']) / matrix['qushi_shop_item_avg_cnt_month']

def select_trend3(row):
    for i in lags:
        if pd.notnull(row['delta2_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta2_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta2_cnt_month_lag'] = matrix.apply(select_trend3, axis=1)
matrix['delta2_cnt_month_lag'] = matrix['delta2_cnt_month_lag'].astype(np.float16)

features_to_drop = ['qushi_shop_item_avg_cnt_month','qushi_date_shop_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['delta2_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

MemoryError: Unable to allocate 1.64 GiB for an array with shape (79, 11128050) and data type float16

In [19]:
# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

In [20]:
matrix.shape

(6639294, 74)

In [12]:
"""建模"""
trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 3000,
    'max_depth': 7,
    'num_leaves': 200,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.005,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.352051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12690
[LightGBM] [Info] Number of data points in the train set: 6186922, number of used features: 78
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.288852
[1]	training's rmse: 1.18588	valid_1's rmse: 1.13488
[2]	training's rmse: 1.18345	valid_1's rmse: 1.13318
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's rmse: 1.18088	valid_1's rmse: 1.13125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's rmse: 1.17852	valid_1's rmse: 1.12955
[LightGBM] [Warning] No further splits with positive

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	training's rmse: 1.05511	valid_1's rmse: 1.04259
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	training's rmse: 1.0537	valid_1's rmse: 1.04162
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	training's rmse: 1.05221	valid_1's rmse: 1.04059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	training's rmse: 1.05074	valid_1's rmse: 1.03959
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	training's rmse: 1.04934	valid_1's rmse: 1.03857
[77]	training's rmse: 1.04792	valid_1's rmse: 1.03764
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	training's rmse: 1.0464	valid_1's rmse: 1.03651
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	training's rmse: 1.04506	valid_1's rmse: 1.03559
[80]	training's rmse: 1.0437	valid_1's rmse: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	training's rmse: 0.97722	valid_1's rmse: 0.990463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[142]	training's rmse: 0.976386	valid_1's rmse: 0.989956
[143]	training's rmse: 0.975569	valid_1's rmse: 0.989425
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[144]	training's rmse: 0.974724	valid_1's rmse: 0.988895
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[145]	training's rmse: 0.9739	valid_1's rmse: 0.988373
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[146]	training's rmse: 0.973064	valid_1's rmse: 0.987898
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[147]	training's rmse: 0.972232	valid_1's rmse: 0.987404
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[148]	training's rmse: 0.97139	valid_1's rmse: 0.986893
[149]	training's rmse: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[209]	training's rmse: 0.929125	valid_1's rmse: 0.959556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[210]	training's rmse: 0.928564	valid_1's rmse: 0.959197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	training's rmse: 0.928037	valid_1's rmse: 0.958892
[212]	training's rmse: 0.927466	valid_1's rmse: 0.958546
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	training's rmse: 0.926858	valid_1's rmse: 0.958114
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[214]	training's rmse: 0.926329	valid_1's rmse: 0.957831
[215]	training's rmse: 0.92582	valid_1's rmse: 0.957564
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[216]	training's rmse: 0.925312	valid_1's rmse: 0.957289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[217]	training's rms

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[277]	training's rmse: 0.899133	valid_1's rmse: 0.942031
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	training's rmse: 0.898723	valid_1's rmse: 0.941814
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	training's rmse: 0.898412	valid_1's rmse: 0.941669
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[280]	training's rmse: 0.898064	valid_1's rmse: 0.941522
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[281]	training's rmse: 0.897697	valid_1's rmse: 0.941356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[282]	training's rmse: 0.897349	valid_1's rmse: 0.94115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[283]	training's rmse: 0.897063	valid_1's rmse: 0.941012
[284]	training's rmse: 0.896677	valid_1's rmse: 0.940755
[LightGBM] [Warning]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2715]	training's rmse: 0.782177	valid_1's rmse: 0.905399
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2716]	training's rmse: 0.782171	valid_1's rmse: 0.905397
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2717]	training's rmse: 0.782161	valid_1's rmse: 0.905395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2718]	training's rmse: 0.78215	valid_1's rmse: 0.905386
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2719]	training's rmse: 0.782142	valid_1's rmse: 0.905384
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2720]	training's rmse: 0.782114	valid_1's rmse: 0.90538
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2721]	training's rmse: 0.782105	valid_1's rmse: 0.905367
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2777]	training's rmse: 0.781525	valid_1's rmse: 0.905465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2778]	training's rmse: 0.781518	valid_1's rmse: 0.905466
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2779]	training's rmse: 0.781509	valid_1's rmse: 0.905471
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2780]	training's rmse: 0.781492	valid_1's rmse: 0.90547
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2781]	training's rmse: 0.781474	valid_1's rmse: 0.905476
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2782]	training's rmse: 0.781467	valid_1's rmse: 0.905478
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2783]	training's rmse: 0.781452	valid_1's rmse: 0.905474
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2839]	training's rmse: 0.780863	valid_1's rmse: 0.905589
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2840]	training's rmse: 0.780858	valid_1's rmse: 0.905588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2841]	training's rmse: 0.780843	valid_1's rmse: 0.905576
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2842]	training's rmse: 0.78083	valid_1's rmse: 0.905572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2843]	training's rmse: 0.780824	valid_1's rmse: 0.905572
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2844]	training's rmse: 0.78082	valid_1's rmse: 0.905568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2845]	training's rmse: 0.780807	valid_1's rmse: 0.905567
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2901]	training's rmse: 0.780235	valid_1's rmse: 0.905523
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2902]	training's rmse: 0.78022	valid_1's rmse: 0.905535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2903]	training's rmse: 0.780213	valid_1's rmse: 0.905534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2904]	training's rmse: 0.780196	valid_1's rmse: 0.905525
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2905]	training's rmse: 0.78019	valid_1's rmse: 0.905528
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2906]	training's rmse: 0.780176	valid_1's rmse: 0.905524
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2907]	training's rmse: 0.780168	valid_1's rmse: 0.905534
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2963]	training's rmse: 0.779539	valid_1's rmse: 0.905506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2964]	training's rmse: 0.779527	valid_1's rmse: 0.905519
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2965]	training's rmse: 0.779518	valid_1's rmse: 0.905514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2966]	training's rmse: 0.77951	valid_1's rmse: 0.905526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2967]	training's rmse: 0.779497	valid_1's rmse: 0.905527
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2968]	training's rmse: 0.779483	valid_1's rmse: 0.905525
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2969]	training's rmse: 0.779476	valid_1's rmse: 0.905526
[LightGBM] [Warning] No further splits with positive gain, best gain: 